In [53]:
# !python -m pip install geopandas
import pandas as pd
import numpy as np
import geopandas as gpd

In [67]:
#reading rheast and extracting districts
rheast= pd.read_csv('data/rheas/maize_zmb_n_25_tamsat_cul_assimila_on_fertilizer5.csv')
pre=pd.read_csv('data/preharvest/preharvest.csv')
post=pd.read_csv('data/postharvest/moa_data.csv')
post['province']=post['province'].replace(['North Western'], 'North-Western')

# rheast.rename(columns={'cname': 'district'},inplace=True)

# #Add a year column to preharvest data and calculate yield
# # pre['year']=pre['season'].str.extract(r'(\d{4})')
pre['year']=pre[' Year']
pre['district']=pre['cname']
rheast['district']=rheast['cname']


pre['yield']=pre[' Yield (MT/Ha)']

post['year']=post['season'].str.extract(r'(\d{4})')
post['yield']=post['production']/post['hactares']
#
# # #Extracting Districts
pre_list_districts= pre['district'].to_list()
pre_set_districts=set(pre_list_districts)
pre_districts=list(pre_set_districts)
rheast_list_districts=rheast['district'].to_list()
rheast_set_districts=set(rheast_list_districts)
rheas_districts=list(rheast_set_districts)

In [68]:
#Finding common districts in rheas and preharvest rheast
districts=[]
for i in pre_districts:
    if i in rheas_districts:
        districts.append(i)

missing_districts=[]
for j in pre_districts:
    if j not in  rheas_districts:
        missing_districts.append(j)
# missing_districts
#
# print(rheast[rheast['district'].str.contains('Mi')])
# mask = np.column_stack(rheast['district'].str.contains(r"^\Ndola", na=False) for col in df])
# df.loc[mask.any(axis=1)]
# len(pre_districts)
# if rheast['district'].str.contains('Kapiri').any():


In [69]:
#changing date columns
rheast['planting']=pd.to_datetime(rheast['planting'], format='ISO8601')
rheast['harvest'] = pd.to_datetime(rheast['harvest'], format='ISO8601')
rheast['year'] = rheast['planting'].dt.year
rheast['yield']=rheast['gwad']/1000


In [77]:
#Derive seasons from the data
seasons = [1, 1, 1, 1,1, 0, 0, 0, 0, 1, 1, 1]
month_to_season=dict(zip(range(1,13), seasons))
rheast['season']= rheast['planting'].dt.month.map(month_to_season)

# Extract data only for the planting season
rheast_model= rheast.loc[rheast['season']==1]
rheast_data=rheast_model.groupby(['district','year', 'season'], as_index=False).agg({'yield': 'mean'})
#Filter rheast to merge preharvest i.e exclude 2006, 2007, 2011, 2013, and 2015
rheast_data1=rheast_data.loc[(rheast_data['year']>=2000) & (rheast_data['year']<=2016)]
rheast_data2=rheast_data1.drop(rheast_data1[(rheast_data1['year']==2006)
                                            |(rheast_data1['year']==2007)|(rheast_data1['year']==2011)
                       |(rheast_data1['year']==2013)|(rheast_data1['year']==2015)].index)
# rheast_data3=rheast_data2.reset_index(drop=True, inplace=True)

# (dataframe.drop(dataframe[dataframe['cost'] > 300].index))

pre1=pre.groupby(['district', 'year'], as_index=False).agg({'yield': 'mean'})
# post1=post.groupby(['province', 'year'], as_index=False).agg({'yield': 'mean'})

# pre1.to_csv('zambia_preharvest.csv')
# pre.loc[pre['district']=='Mpongwe']

# d=df.reset_index(drop=True, inplace=True)
# rheast_data[['district', 'year', 'yield']].to_csv('rheas_dssat_model.csv')


In [71]:

#LOOPING through the districts
# pre=pre.loc[pre['district']=='Chadiza'].reset_index(drop=True)
# rheast=rheast_data.loc[rheast_data['district']=='Chadiza'].reset_index(drop=True)
# #  pearson's correlation
# corr=pre['yield'].corr(rheast['yield'])


distric_metrics1=[]
for i in districts:
    pre2=pre1.loc[pre1['district']==i].reset_index(drop=True)
    rheast_data3=rheast_data2.loc[rheast_data2['district']==i].reset_index(drop=True)
    #  pearson's correlation
    corr=rheast_data3['yield'].corr(pre2['yield'])
    # # #root mean square error
    rmse = np.sqrt(((rheast_data3['yield']-pre2['yield'])**2).mean())
    # # #unbiased RMSE
    ubrmse = np.sqrt((((rheast_data3['yield']-rheast_data3['yield'].mean())
                       -(pre2['yield']-pre2['yield'].mean()))**2).mean())
    # #bias
    bias = (rheast_data3['yield']-pre2['yield']).mean()
    # for i in districts:
    # metrics.extend([i, corr])
    distric_metrics1.extend([[i, corr,rmse,ubrmse,bias]])
    # metrics.extend([i, corr])
distric_metrics1
# pre=pre.loc[pre['district']=='Kalomo'].reset_index()
# #  correlation
# corr=pre['yield'].corr(rheast['yield'])
# pre
# dssat=dssat.loc[(dssat['year']==2011) | (dssat['year']==2012) | (dssat['year']==2014)]

# post['yield']=post['production']/post['hactares']
# dssat=dssat.loc[(dssat['year']==2011) | (dssat['year']==2012) | (dssat['year']==2014)]
# post
# post=post.reset_index()
# # dssat=dssat.reset_index()

# # correlation
# # dssat['yield'].corr(post['yield'])
# #rmse
# # pearson's correlation
# corr = dssat['yield'].corr(post['yield']) 
# # #root mean square error
# rmse = np.sqrt(((dssat['yield']-post['yield'])**2).mean())
# post['yield']
# # #unbiased RMSE
# ubrmse = np.sqrt((((dssat['yield']-dssat['yield'].mean())-(post['yield']-post['yield'].mean()))**2).mean())

# # #bias
# bias = (dssat['yield']-post['yield']).mean()

# # print(corr,rmse,ubrmse,bias)
district_statistics1=pd.DataFrame(distric_metrics1)
district_statistics1=district_statistics1.rename({0:'District',1: 'correlation',
                                                  2: 'rmse', 3:'ubrmse', 4:'bias'}, axis=1)
# stats.to_csv('preharvest_dssat_distric_level_metrics.csv')
# stats
# districts
district_statistics1#.to_csv('zambia_preharvest_rheas__district_level_metrics.csv')

,District,correlation,rmse,ubrmse,bias
0,Kasempa,-0.009733,1.051546,0.363744,0.986636
1,Ndola,-0.424639,1.187962,0.491904,1.081346
2,Lufwanyama,-0.466360,1.789755,0.422761,1.739150
3,Kaoma,0.346045,2.401205,0.368047,2.372880
4,Nakonde,0.390392,1.301039,0.391809,1.240640
...,...,...,...,...,...
66,Kalulushi,-0.504218,1.071287,0.396196,0.995350
67,Nyimba,-0.111766,2.241244,0.321759,2.218035
68,Chililabombwe,-0.445798,1.535218,0.370945,1.489756
69,Kalomo,-0.011299,1.678922,0.587922,1.572619


In [72]:
# Province Level
#Read shapefiles and rename columns
zambia_district_shp = gpd.read_file('../zambia/data/shp/zambia_districts.shp')
zambia_shp = gpd.read_file('../zambia/data/shp/gadm36_ZMB_0.shp')
zambia_province_shp = gpd.read_file('../zambia/data/shp/gadm36_ZMB_1.shp')
zambia_district_shp .rename(columns={'DISTRICT':'district', 'PROVINCE':'province'}, inplace=True)
# merge the yields and shapefiles by provinces
pre_shp= pd.merge(zambia_district_shp, pre1, on="district")
post_shp= pd.merge(zambia_district_shp, post, on="province")

rheast_shp= pd.merge(zambia_district_shp, rheast_data2, on="district")
# rheasc_shp= pd.merge(zambia_district_shp, rheast_data2 on="p")
pre_province=pre_shp.groupby(['province', 'year'], as_index=False).agg({'yield': 'mean'})
post_province=post_shp.groupby(['province', 'year'], as_index=False).agg({'yield': 'mean'})
rheast_province=rheast_shp.groupby(['province', 'year'], as_index=False).agg({'yield': 'mean'})
rheast_province

#Extracting Provinces
pre_list_provinces= pre_province['province'].to_list()
pre_set_provinces=set(pre_list_provinces)
pre_provinces=list(pre_set_provinces)
rheast_list_provinces=rheast_province['province'].to_list()
rheast_set_provinces=set(rheast_list_provinces)
rheast_provinces=list(rheast_set_provinces)

#list of provinces
provinces=[]
for i in pre_provinces:
    if i in rheast_provinces:
        provinces.append(i)

In [73]:
province_metrics1=[]
for i in provinces:
    pre_province1=pre_province.loc[pre_province['province']==i].reset_index(drop=True)
    rheast_province1=rheast_province.loc[rheast_province['province']==i].reset_index(drop=True)  
    #  pearson's correlation
    corr=rheast_province['yield'].corr(pre_province1['yield'])

    # #root mean square error
    rmse = np.sqrt(((rheast_province1['yield']-pre_province1['yield'])**2).mean())
    # #unbiased RMSE
    ubrmse = np.sqrt((((rheast_province1['yield']-rheast_province1['yield'].mean())
                       -(pre_province1['yield']-pre_province1['yield'].mean()))**2).mean())
    # #bias
    bias = (rheast_province1['yield']-pre_province1['yield']).mean()
    # for i in districts:
    # metrics.extend([i, corr])
    province_metrics1.extend([[i, corr,rmse,ubrmse,bias]])
    # metrics.extend([i, corr])

province_statistics1=pd.DataFrame(province_metrics1)
province_statistics1=province_statistics1.rename({0:'province',1: 'correlation',2: 
                                                  'rmse', 3:'ubrmse', 4:'bias'}, axis=1)
# province_statistics1.to_csv('preharvest_dssat_province_level_metrics.csv')
province_statistics1#.to_csv('zambia_preharvest_rheas__province_level_metrics.csv')

,province,correlation,rmse,ubrmse,bias
0,Southern,-0.070499,2.337667,0.468478,2.290248
1,Luapula,-0.256000,1.253491,0.204256,1.236770
2,Copperbelt,-0.692428,1.158466,0.271699,1.126179
3,Western,-0.235645,2.990499,0.268287,2.978451
4,North-Western,-0.009190,1.457863,0.224911,1.440439
5,Eastern,-0.301348,1.717112,0.268852,1.695939
6,Northern,0.203799,1.260999,0.185928,1.247261
7,Lusaka,-0.119953,1.700875,0.685856,1.556470
8,Muchinga,0.468040,0.655288,0.163197,0.634687
9,Central,-0.087968,1.103215,0.469659,0.998255


In [79]:
# Postharvest and rheas common years 
post_list_years= post['year'].to_list()
years_set=set(post_list_years)
post_harvest_years=list(years_set)
post_harvest_years

# df1=rheast_province.loc[rheast_province['year']==2016]
rheast_province2=pd.DataFrame(columns =[rheast_province.columns])
for i in post_harvest_years:
    df=rheast_province.loc[rheast_province['year']==i].reset_index(drop=True)
    rheast_province2=pd.concat([df])

# for i in post_harvest_years:
#     rheast_province2=pd.concat(rheast_province.loc[(rheast_province['year']==i)], axis=1)
    
# rheast_list_provinces=rheast_province['province'].to_list()
# rheast_set_provinces=set(rheast_list_provinces)
# rheast_provinces=list(rheast_set_provinces)

In [75]:
# Postharvest and rheas metrics


province_metrics2=[]
for i in provinces:
    post_province1=post_province.loc[post_province['province']==i].reset_index(drop=True)
    rheast_province2=rheast_province2.loc[rheast_province['province']==i].reset_index(drop=True)

    #  pearson's correlation
    corr=rheast_province2['yield'].corr(post_province1['yield'])

    # #root mean square error
    rmse = np.sqrt(((rheast_province2['yield']-post_province1['yield'])**2).mean())
    # #unbiased RMSE
    ubrmse = np.sqrt((((rheast_province2['yield']-rheast_province2['yield'].mean())
                       -(post_province1['yield']-post_province1['yield'].mean()))**2).mean())
    # #bias
    bias = (rheast_province2['yield']-post_province1['yield']).mean()
    # for i in districts:
    # metrics.extend([i, corr])
    province_metrics2.extend([[i, corr,rmse,ubrmse,bias]])
    # metrics.extend([i, corr])

province_statistics2=pd.DataFrame(province_metrics2)
province_statistics2=province_statistics2.rename({0:'province',1: 
                                                  'correlation',2: 'rmse', 3:'ubrmse', 4:'bias'}, axis=1)
# province_statistics.to_csv('preharvest_dssat_province_level_metrics.csv')
province_statistics2#.to_csv('zambia_post_harvest_province_level_metrics.csv')
rheast_province2
# post_province1

,province,year,yield


In [76]:
# pre_province1
rheast_province
# post_province1


,province,year,yield
0,Central,2000,3.507455
1,Central,2001,3.482528
2,Central,2002,3.521872
3,Central,2003,3.379310
4,Central,2004,3.693470
...,...,...,...
115,Western,2009,3.763253
116,Western,2010,3.617881
117,Western,2012,3.694521
118,Western,2014,3.644344
